In [1]:
import pandas as pd
import numpy as np

In [25]:
def get_industry(sic_code):
    """
    Returns the corresponding industry column name based on the SIC code.
    
    Parameters:
    sic_code (int): A four-digit SIC code.
    
    Returns:
    str: Corresponding industry column name.
    """
    # Define SIC code ranges and their corresponding column names
    sic_to_column = [
        (range(100, 1000), "Agric"),
        (range(1000, 1300), "Mines"),
        (range(1300, 1400), "Oil"),
        (range(1400, 1500), "Stone"),
        (range(1500, 1800), "Cnstr"),
        (range(2000, 2100), "Food"),
        (range(2100, 2200), "Smoke"),
        (range(2200, 2300), "Txtls"),
        (range(2300, 2400), "Apprl"),
        (range(2400, 2500), "Wood"),
        (range(2500, 2600), "Chair"),
        (range(2600, 2662), "Paper"),
        (range(2700, 2800), "Print"),
        (range(2800, 2900), "Chems"),
        (range(2900, 3000), "Ptrlm"),
        (range(3000, 3100), "Rubbr"),
        (range(3100, 3200), "Lethr"),
        (range(3200, 3300), "Glass"),
        (range(3300, 3400), "Metal"),
        (range(3400, 3500), "MtlPr"),
        (range(3500, 3600), "Machn"),
        (range(3600, 3700), "Elctr"),
        (range(3700, 3800), "Cars"),
        (range(3800, 3880), "Instr"),
        (range(3900, 4000), "Manuf"),
        (range(4000, 4800), "Trans"),
        (range(4800, 4830), "Phone"),
        (range(4830, 4900), "TV"),
        (range(4900, 4950), "Utils"),
        (range(4950, 4960), "Garbg"),
        (range(4960, 4970), "Steam"),
        (range(4970, 4980), "Water"),
        (range(5000, 5200), "Whlsl"),
        (range(5200, 6000), "Rtail"),
        (range(6000, 7000), "Money"),
        (range(7000, 9000), "Srvc"),
        (range(9000, 10000), "Govt"),
    ]
    
    # Match the SIC code to the corresponding column name
    for code_range, column_name in sic_to_column:
        if sic_code in code_range:
            return column_name.strip()
    
    return "Other"

In [26]:
df_msf = pd.read_csv('MSF_1996_2023.csv')
df_msf['date'] = pd.to_datetime(df_msf['date']).dt.to_period('M')
df_msf['RET'] = pd.to_numeric(df_msf['RET'], errors='coerce')
# df_msf = df_msf[df_msf['RET'].notna()]
df_msf['vwretd'] = pd.to_numeric(df_msf['vwretd'], errors='coerce')

df_msf['SICCD'] = pd.to_numeric(df_msf['SICCD'], errors='coerce')
# group by PERMNO and calculate covariates
def calc_covariates(group):
    # Calculate the rolling beta using covariance and variance
    group['beta'] = group['RET'].rolling(window=12).cov(group['vwretd']) / group['vwretd'].rolling(window=12).var()
    return group
df_msf = df_msf.groupby('PERMNO').apply(calc_covariates)

df_msf['industry'] = df_msf['SICCD'].apply(get_industry)

df_msf['mkt_cap'] = df_msf['SHROUT'] * df_msf['PRC'].abs()

df_msf = df_msf.reset_index(drop=True)
df_msf = df_msf.dropna()
df_msf.columns

/var/folders/d8/y1wr969x5h7_cbmcpr1dlxt80000gn/T/ipykernel_94580/1410109383.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_msf = pd.read_csv('MSF_1996_2023.csv')
/var/folders/d8/y1wr969x5h7_cbmcpr1dlxt80000gn/T/ipykernel_94580/1410109383.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_msf = df_msf.groupby('PERMNO').apply(calc_covariates)


Index(['PERMNO', 'date', 'SHRCD', 'SICCD', 'TICKER', 'COMNAM', 'PERMCO',
       'CUSIP', 'BIDLO', 'ASKHI', 'PRC', 'VOL', 'RET', 'BID', 'ASK', 'SHROUT',
       'RETX', 'vwretd', 'beta', 'industry'],
      dtype='object')

In [37]:
df_ind = pd.read_csv('38_Industry_Portfolios.csv', skiprows=11, nrows=1170, index_col=0)
# Missing data are indicated by -99.99 or -999.
df_ind = df_ind.reset_index().rename(columns={"index": "date"})
df_ind['date'] = pd.to_datetime(df_ind['date'], format='%Y%m').dt.to_period('M')

# Iterate through each row in df_msf and assign the corresponding industry return
industry_columns = list(df_ind.columns[1:])  # Exclude 'date' column
df_msf['Industry_RET'] = df_msf.apply(
    lambda row: df_ind.loc[df_ind['date'] == row['date'], row['industry']].values[0]
    if row['industry'] in industry_columns else None, axis=1
    )

,date,Agric,Mines,Oil,Stone,Cnstr,Food,Smoke,Txtls,Apprl,...,Utils,Garbg,Steam,Water,Whlsl,Rtail,Money,Srvc,Govt,Other
0,1926-07,2.37,2.64,-2.69,9.64,-8.21,0.03,1.29,0.39,1.28,...,7.04,-99.99,-99.99,-99.99,-23.79,0.14,-0.02,3.79,-99.99,-5.95
1,1926-08,2.23,0.67,3.78,0.77,7.37,3.07,6.50,7.97,1.54,...,-1.69,-99.99,-99.99,-99.99,5.39,-0.72,4.47,-0.20,-99.99,0.00
2,1926-09,-0.57,-0.22,-5.92,7.47,-11.76,1.63,1.26,2.30,-2.42,...,2.04,-99.99,-99.99,-99.99,-7.87,0.22,-1.61,5.56,-99.99,0.00
3,1926-10,-0.46,-0.34,-1.98,-2.78,-8.59,-3.67,1.06,1.00,-0.68,...,-2.63,-99.99,-99.99,-99.99,-15.38,-2.28,-5.51,-4.47,-99.99,0.00
4,1926-11,6.75,1.58,0.91,17.23,2.25,6.28,4.55,3.10,4.48,...,3.71,-99.99,-99.99,-99.99,4.67,6.43,2.34,2.22,-99.99,0.00


In [48]:
df = df_msf[['PERMNO','date','SICCD','RET','PRC','SHROUT','beta','industry','mkt_cap','Industry_RET']]
df.head()

,PERMNO,date,SICCD,RET,PRC,SHROUT,beta,industry,mkt_cap,Industry_RET
0,10001,1996-12,4920.0,-0.059429,8.125,2357.0,0.354364,Utils,19150.625,0.13
1,10001,1997-01,4920.0,0.061538,8.625,2357.0,0.535780,Utils,20329.125,0.92
2,10001,1997-02,4920.0,0.000000,8.625,2357.0,0.521469,Utils,20329.125,-0.57
3,10001,1997-03,4920.0,0.012174,8.625,2357.0,0.360518,Utils,20329.125,-2.78
4,10001,1997-04,4920.0,0.000000,8.625,2357.0,0.397868,Utils,20329.125,-1.39


In [50]:
def calculate_factors(df):
    # Ensure the data is sorted by PERMNO and date
    df = df.sort_values(by=['PERMNO', 'date']).reset_index(drop=True)
    
    # Create placeholder columns for the factors
    df['Short_term_reversal'] = None
    df['Momentum'] = None
    df['Long_term_reversal'] = None
    
    # Group by PERMNO and calculate factors for each group
    grouped = df.groupby('PERMNO')
    
    for name, group in grouped:
        # Ensure the group is sorted by date
        group = group.sort_values(by='date')

        # Calculate Short-term reversal
        short_term_reversal = group['RET'].shift(1)

        # Calculate Momentum (rolling 12-month sum excluding prior month)
        momentum = group['RET'].rolling(window=12, min_periods=12).sum().shift(1) - group['RET'].shift(1)

        # Calculate Long-term reversal (rolling 60-month sum excluding prior year)
        long_term_reversal = (
            group['RET'].rolling(window=60, min_periods=60).sum().shift(1) -
            group['RET'].rolling(window=12, min_periods=12).sum().shift(1)
        )

        # Assign the calculated factors back to the original DataFrame
        df.loc[group.index, 'Short_term_reversal'] = short_term_reversal
        df.loc[group.index, 'Momentum'] = momentum
        df.loc[group.index, 'Long_term_reversal'] = long_term_reversal
    
    return df

# Apply the function to your DataFrame
df = calculate_factors(df)
df.head()

,PERMNO,date,SICCD,RET,PRC,SHROUT,beta,industry,mkt_cap,Industry_RET,Short_term_reversal,Momentum,Long_term_reversal
0,10001,1996-12,4920.0,-0.059429,8.125,2357.0,0.354364,Utils,19150.625,0.13,NaN,NaN,NaN
1,10001,1997-01,4920.0,0.061538,8.625,2357.0,0.535780,Utils,20329.125,0.92,-0.059429,NaN,NaN
2,10001,1997-02,4920.0,0.000000,8.625,2357.0,0.521469,Utils,20329.125,-0.57,0.061538,NaN,NaN
3,10001,1997-03,4920.0,0.012174,8.625,2357.0,0.360518,Utils,20329.125,-2.78,0.0,NaN,NaN
4,10001,1997-04,4920.0,0.000000,8.625,2357.0,0.397868,Utils,20329.125,-1.39,0.012174,NaN,NaN


In [53]:
df = df.dropna()
df.to_csv('msf_data.csv', index=False)

In [143]:
# merge with Fama-French factors

df = pd.read_csv('msf_data.csv')
df['date'] = pd.to_datetime(df['date']).dt.to_period('M')
ff_factors = pd.read_csv('merged_factors.csv')
ff_factors = ff_factors.dropna()
ff_factors['date'] = pd.to_datetime(ff_factors['Date']).dt.to_period('M')
df= df.merge(ff_factors, on='date', how='left')
df.drop(columns=['Date'], inplace=True)
df.head()

,PERMNO,date,SICCD,RET,PRC,SHROUT,beta,industry,mkt_cap,Industry_RET,...,Long_term_reversal,Mkt_RF,SMB,HML,RMW,CMA,RF,Mom,LT_Rev,ST_Rev
0,10001,2001-12,4920.0,0.006957,11.45,2566.0,-0.072822,Utils,29380.70,4.04,...,0.331275,1.60,5.16,0.84,0.34,-0.26,0.15,0.07,1.11,1.68
1,10001,2002-01,4920.0,-0.013100,11.30,2566.0,-0.067655,Utils,28995.80,-4.34,...,0.423384,-1.44,1.26,3.44,4.69,2.86,0.14,3.75,3.30,2.13
2,10001,2002-02,4920.0,-0.053097,10.70,2569.0,-0.134151,Utils,27488.30,-0.37,...,0.374667,-2.29,-0.36,2.16,8.07,5.11,0.13,6.82,4.07,0.05
3,10001,2002-03,4920.0,-0.015888,10.40,2571.0,-0.122017,Utils,26738.40,10.28,...,0.362009,4.24,4.25,1.06,-1.78,0.59,0.13,-1.64,2.08,2.75
4,10001,2002-04,4920.0,-0.043269,9.95,2571.0,0.065439,Utils,25581.45,-0.41,...,0.388297,-5.20,6.72,3.88,4.56,5.37,0.15,7.63,5.96,2.11


In [204]:
df_compustat = pd.read_csv('financial_ratios_data.csv')
df_compustat.columns

/var/folders/d8/y1wr969x5h7_cbmcpr1dlxt80000gn/T/ipykernel_94580/3003349922.py:1: DtypeWarning: Columns (58,67,71,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df_compustat = pd.read_csv('financial_ratios_data.csv')


Index(['Unnamed: 0', 'datadate', 'gvkey', 'fyearq', 'fqtr', 'fyr', 'indfmt',
       'consol', 'popsrc', 'datafmt',
       ...
       'TXTQH', 'DOQH', 'DVQH', 'EPS', 'EPS_lag', 'EPS_std', 'SUE',
       'Scaled_NOA', 'Share_Issuance', 'Z_Score'],
      dtype='object', length=137)

In [206]:
df_compustat.drop(['Unnamed: 0','gvkey', 'Short_Term_Reversal','Momentum','Long_Term_Reversal', 'PRC', 'VOL', 'RET', 'BID', 'ASK',
       'SHROUT', 'RETX', 'vwretd','SICCD', 'LINKPRIM', 'LIID', 'LINKTYPE', 'LPERMNO', 'SHRCD',
       'SICCD', 'TICKER', 'COMNAM', 'PERMCO', 'CUSIP', 'BIDLO', 'ASKHI',
       'LPERMCO', 'LINKDT', 'LINKENDDT'], axis=1, inplace=True)

# Perform calculations first
df_compustat['PPENTQH'] = df_compustat['atq'] - df_compustat['invtq']  # Net Property, Plant, and Equipment (approximation)
df_compustat['AOQH'] = df_compustat['atq'] - df_compustat['lctq'] - df_compustat['invtq']  # Total Other Assets (approximation)
df_compustat['DLTTQH'] = df_compustat['ltq'] - df_compustat['lctq']  # Total Long-term Debt
df_compustat['ICAPTQH'] = df_compustat['ceqq'] + df_compustat['ltq']  # Total Invested Capital

df_compustat['date'] = pd.to_datetime(df_compustat['date']).dt.to_period('M')

df_compustat.columns

Index(['datadate', 'fyearq', 'fqtr', 'fyr', 'indfmt', 'consol', 'popsrc',
       'datafmt', 'tic', 'cusip',
       ...
       'TXTQH', 'DOQH', 'DVQH', 'EPS', 'EPS_lag', 'EPS_std', 'SUE',
       'Scaled_NOA', 'Share_Issuance', 'Z_Score'],
      dtype='object', length=109)

In [228]:
# merge df and df_compustat based on PERMNO and date, left join
df = df.drop_duplicates(subset=['PERMNO', 'date'])
df_compustat = df_compustat.drop_duplicates(subset=['PERMNO', 'date'])

df_merged = df.merge(df_compustat, how='left', on=['PERMNO', 'date'])

print(len(df_merged))
df_merged.head()


553947


,PERMNO,date,SICCD,RET,PRC,SHROUT,beta,industry,mkt_cap,Industry_RET,...,TXTQH,DOQH,DVQH,EPS,EPS_lag,EPS_std,SUE,Scaled_NOA,Share_Issuance_y,Z_Score
0,10001,2001-12,4920.0,0.006957,11.45,2566.0,-0.072822,Utils,29380.70,4.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10001,2002-01,4920.0,-0.013100,11.30,2566.0,-0.067655,Utils,28995.80,-4.34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10001,2002-02,4920.0,-0.053097,10.70,2569.0,-0.134151,Utils,27488.30,-0.37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10001,2002-03,4920.0,-0.015888,10.40,2571.0,-0.122017,Utils,26738.40,10.28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10001,2002-04,4920.0,-0.043269,9.95,2571.0,0.065439,Utils,25581.45,-0.41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [229]:
# Forward-fill missing values grouped by PERMNO
df_merged = df_merged.sort_values(by=['PERMNO', 'date'])  # Sort by PERMNO and date
df_merged = df_merged.groupby('PERMNO').apply(lambda group: group.ffill()).reset_index(drop=True)
df_merged = df_merged.dropna()
print(len(df_merged))
df_merged.head()

/var/folders/d8/y1wr969x5h7_cbmcpr1dlxt80000gn/T/ipykernel_94580/2962279162.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_merged = df_merged.groupby('PERMNO').apply(lambda group: group.ffill()).reset_index(drop=True)
/var/folders/d8/y1wr969x5h7_cbmcpr1dlxt80000gn/T/ipykernel_94580/2962279162.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_merged = df_merged.groupby('PERMNO').apply(lambda group: group.ffill()).reset_index(drop=True)
/var/folders/d8/y1wr969x5h7_cbmcpr1dlxt80000gn/T/ipykernel_94580/2962279162.py:3: FutureWarning: Downcasting

334291


,PERMNO,date,SICCD,RET,PRC,SHROUT,beta,industry,mkt_cap,Industry_RET,...,TXTQH,DOQH,DVQH,EPS,EPS_lag,EPS_std,SUE,Scaled_NOA,Share_Issuance_y,Z_Score
561,10028,2002-09,5944.0,0.000000,1.170,4914.0,1.048013,Rtail,5749.380,-9.95,...,-0.015,1.062,0.17,-0.000006,1.569355e-05,0.000013,-1.632882,0.999794,0.000023,562.662866
562,10028,2002-10,5944.0,0.042735,1.220,4914.0,0.987147,Rtail,5995.080,5.82,...,-0.015,1.062,0.17,-0.000006,1.569355e-05,0.000013,-1.632882,0.999794,0.000023,562.662866
563,10028,2002-11,5944.0,-0.032787,1.180,4914.0,1.240796,Rtail,5798.520,2.97,...,-0.015,1.062,0.17,-0.000006,1.569355e-05,0.000013,-1.632882,0.999794,0.000023,562.662866
564,10028,2002-12,5944.0,-0.097458,-1.065,4913.0,0.621448,Rtail,5232.345,-7.83,...,0.289,1.062,0.17,0.000111,-6.557377e-07,0.000041,2.718924,0.996211,-0.000068,-540.442598
565,10028,2003-01,5944.0,-0.014085,1.050,4913.0,0.622134,Rtail,5158.650,-5.15,...,0.289,1.062,0.17,0.000111,-6.557377e-07,0.000041,2.718924,0.996211,-0.000068,-540.442598


In [230]:
df_merged['Share_Issuance'] = (
    df_merged['SHROUT'] / df_merged.groupby('PERMNO')['SHROUT'].shift(1) - 1
)

df_merged.rename(columns={'PRC': 'Share_price', 'RET': 'Return', 'SHROUT': 'Number_of_shares_outstanding'}, inplace=True)

df_merged['DVPQH'] = df_merged['DVQH']/df_merged['PPENTQH']
df_merged = df_merged[dataset_columns]
df_merged.columns

Index(['PERMNO', 'date', 'Return', 'Share_price',
       'Number_of_shares_outstanding', 'beta', 'industry', 'mkt_cap',
       'Industry_RET', 'Short_term_reversal', 'Momentum', 'Long_term_reversal',
       'Mkt_RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'Mom', 'LT_Rev', 'ST_Rev',
       'ACOQH', 'ATQH', 'SEQQH', 'LTQH', 'NOPIQH', 'rectq', 'SALEQH', 'TXTQH',
       'oancfy', 'costat', 'APQH', 'DVPQH', 'CHEQH', 'NIQH', 'PSTKQH',
       'Market_Capitalization', 'Accruals', 'Gross_Profitability',
       'Book_to_Market', 'Asset_Growth', 'Investment_Ratio', 'Leverage',
       'Earnings_Yield', 'Dividend_Price', 'Cash_Flow_Price', 'Illiquidity',
       'ROE', 'ROA', 'Current_Ratio', 'Quick_Ratio', 'Net_Profit_Margin',
       'Gross_Margin', 'ICAPTQH', 'PSTKRQH', 'TEQQH', 'PPENTQH', 'CEQQH',
       'AOQH', 'DLTTQH', 'LOQH', 'LCOQH', 'XIDOQH', 'IBQH', 'IBADJQH',
       'IBCOMQH', 'PIQH', 'DOQH', 'DVQH', 'EPS', 'EPS_lag', 'EPS_std', 'SUE',
       'Share_Issuance', 'Scaled_NOA', 'Z_Score'],
      d

In [232]:
dataset_columns = ['PERMNO', 'date', 'Return', 'Share_price',
       'Number_of_shares_outstanding', 'beta', 'industry', 'mkt_cap',
       'Industry_RET', 'Short_term_reversal', 'Momentum', 'Long_term_reversal',
       'Mkt_RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'Mom', 'LT_Rev', 'ST_Rev',
       'ACOQH', 'ATQH', 'SEQQH', 'LTQH', 'NOPIQH', 'SALEQH', 'TXTQH',
       'APQH', 'DVPQH', 'CHEQH',  'NIQH', 'PSTKQH',
       'Market_Capitalization', 'Accruals', 'Gross_Profitability',
       'Book_to_Market', 'Asset_Growth', 'Investment_Ratio', 'Leverage',
       'Earnings_Yield', 'Dividend_Price', 'Cash_Flow_Price', 'Illiquidity',
       'ROE', 'ROA', 'Current_Ratio', 'Quick_Ratio', 'Net_Profit_Margin',
       'Gross_Margin', 'ICAPTQH', 'PSTKRQH', 'TEQQH',
       'PPENTQH','CEQQH', 'AOQH', 'DLTTQH', 'LOQH','LCOQH',
       'XIDOQH', 'IBQH', 'IBADJQH','IBCOMQH', 'PIQH', 'DOQH', 'DVQH', 'EPS', 'EPS_lag', 'EPS_std', 'SUE',
       'Share_Issuance', 'Scaled_NOA', 'Z_Score']
df_merged = df_merged[dataset_columns]
# df_merged[:1000].to_csv('merged_data.csv', index=False)

In [233]:
df_merged.to_csv('dataset.csv', index=False)

In [ ]:
# Rename columns where applicable
df_compustat.rename(columns={
    'atq': 'ATQH',  # Total Assets
    'ceqq': 'SEQQH',  # Total Stockholders Equity
    'ltq': 'LTQH',  # Total Liabilities
    'pstkq': 'PSTKQH',  # Redeemable Preferred/Preference Stock
    'actq': 'ACOQH',  # Total Other Current Assets
    'cheq': 'CHEQH',  # Cash and Short-term Investments
    'apq': 'APQH',  # Accounts Payable
    'dpactq': 'DVPQH',  # Preferred/Preference Dividends
    'revtq': 'SALEQH',  # Sales/Turnover (Net)
    'oibdpq': 'NOPIQH',  # Non-operating Income (Expense)
    'ibq': 'NIQH',  # Net Income (Loss)
    'capxy': 'TXTQH',  # Total Income Taxes (approximation)
}, inplace=True)